In [ ]:
from typing import List, Dict, Tuple, Optional, Union, Any

import pandas as pd

from prompt_template_collection import PromptTemplate
from doraemon import Doraemon
from relaxed_fda import RelaxedFDA
from metrics_collection import MetricsHelper

task_name = "GSM8K"
hp_m = None # -> SoTs 4,6,8,10(default),12
hp_k = 7 # -> cluster 1,3,4,5,7
# Set up logger
logger = Doraemon.get_logger(name=task_name, logfile=f"relaxed-fda_on_{task_name}.log")


df=pd.read_pickle('/kaggle/input/building-gsm8k-sots-dataset/sots.pkl')
df=df.rename(columns={'question':'query', 'reason':'r_s', 'ground_truth':'g_t','temperature':'t_p'})
if hp_m:
    df = df.groupby('query').head(hp_m).reset_index(drop=True)
df.to_pickle('sots_df.pkl')
grouped_data = RelaxedFDA.prepare_dataset(file_path='/kaggle/working/sots_df.pkl', mode='r_s', logger=logger).head(1200)

D: List[Dict] = PromptTemplate.sot_construct_inter_gsm8k()[:1]
op_system_prompt = RelaxedFDA.get_optimize_system_prompt(task_name)
encoder=RelaxedFDA.get_encoder()

In [ ]:
result_pd: pd.DataFrame = RelaxedFDA.evaluate(
    grouped_data,
    logger,
    op_system_prompt,
    encoder,
    D,
    enable_logger_rs=False,
    K=hp_k
)

In [ ]:
MetricsHelper.evaluate(result_pd, logger, log=True)